# Reference 

1. open dart official document
https://dart-fss.readthedocs.io/en/latest/welcome.html

2. 코딩 유치원 
https://coding-kindergarten.tistory.com/85

In [3]:
!pip install opendartreader

In [4]:
import OpenDartReader

In [5]:
# xml 파일을 읽기 위한 라이브러리
# from xml.etree.ElementTree import parse
# 판다스 
import pandas as pd
import os
import dart_fss as dart

My api key :7db60698e63bd621a006025500fe0436ee65cb47

dart 서비스에 올라가 있는 기업들은 비상장 기업들의 재무정보도 올라와 있다. 
우리는 상장사의 재무정보만 보고 싶은 것이기 때문에, 전체 corp_list 중에서 상장기업들만 추출해야 한다.   

상장사 리스트를 얻기 위해 한국거래소 > 기본 통계 > 주식 > 종목정보 > 전종목 기본정보에서 전종목 기본정보를 다운받는다.   
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201040201

In [6]:
# 다운받은 전종목 기본정보 불러오기 
all_stocks = pd.read_excel('./data_5655_20220311.xlsx')

C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [7]:
all_stocks.head(1)

,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7098120009,098120,(주)마이크로컨텍솔루션,마이크로컨텍솔,"Micro Contact Solution Co.,Ltd.",2008/09/23,KOSDAQ,주권,벤처기업부,보통주,500,8312766


dart 패키지에서는 주식종목코드를 사용하는 것이아니라, dart 자체적인 corp_code(기업코드)를 가지고 정보를 불러온다.  
따라서 한국거래소에서 얻은 상장기업 주식코드를 가지고, dart의 데이터에서 상장기업 리스트와, 그들의 corp_code를 추출한다. 
여기서 주의 할 것은, 한국거래소에서 얻은 상장기업 주식코드에는 우선주도 포함된다는 점이다. 상장기업 주식코드중 보통주만 사용해야한다. 

In [8]:
# 상장기업 주식 코드 중 보통주만 걸러내기 
listed_companies = all_stocks[all_stocks['주식종류']=='보통주']

In [9]:
import dart_fss as dart

# Open DART API KEY 설정
api_key='7db60698e63bd621a006025500fe0436ee65cb47'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.corp.get_corp_list()

Output()

Output()

Output()

In [10]:
# 상장 기업들의 기업코드와 회사 이름
listed_companies_corp_code = []
listed_companies_name = []

for stock_code in listed_companies['단축코드'] : 
    #corp_list.find_by_corp_name( ) 함수의 결과 데이터는 [[종목 코드]종목 이름]의 형태
    #이를 벗겨내기 위해서 corp_list.find_by_corp_name( )[0]으로 코딩함 
    try : 
        listed_companies_corp_code.append(corp_list.find_by_stock_code(stock_code).corp_code)
        listed_companies_name.append(corp_list.find_by_stock_code(stock_code).corp_name)
        
    except : 
        print(stock_code)
        

041140
220630
225850
329560
052190
039670
225330
050320
950180
138250
225860
270020
240340
330990
153360
066110
053660


In [11]:
# 에러 확인. 현재 무슨일인지 모르겠지만 주식코드 225330, 씨엠에스에듀주식회사 정보를 얻어올 수 없다
corp_list.find_by_stock_code('041140')

(우선주를 제외한)상장기업들 수

In [12]:
len(listed_companies_name)

2482

In [13]:
corp_code_df = pd.DataFrame({'기업이름':listed_companies_name, '기업코드' : listed_companies_corp_code})
corp_code_df.head(5)

,기업이름,기업코드
0,마이크로컨텍솔,00568188
1,스카이이앤엠,00814786
2,포스코엠텍,00128111
3,AJ네트웍스,00365387
4,AK홀딩스,00125080


In [14]:
# 모든 상장된 기업 리스트 불러오기
corp_list = dart.get_corp_list()

In [15]:
for i in ['삼성전자','현대자동차','KB금융','삼성생명'] : 
    print(corp_code_df[corp_code_df['기업이름']== i])
    

     기업이름      기업코드
774  삼성전자  00126380
       기업이름      기업코드
2413  현대자동차  00164742
    기업이름      기업코드
39  KB금융  00688996
     기업이름      기업코드
770  삼성생명  00126256


## 재무제표 가공

In [17]:
## 삼성, 현차 
for iter_num, corp_code in enumerate(['00126380', '00688996']) :     

    # 2019년 01월 01일 부터 연결재무제표 검색
    # fs = samsung.extract_fs(bgn_de='20120101') 와 동일
    fs = dart.fs.extract(corp_code=corp_code, bgn_de='20200101', report_tp = ['annual','half','quarter'])


    # class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
    bs_df = fs.show('bs', show_class = False, show_concept = True).T.reset_index()
    # level_1은 쓸모없는 설명 열 이므로 제거 
    bs_df.drop(['level_1'],axis=1, inplace=True)
    bs_df.columns = bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
    bs_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
    # 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
    bs_df.rename(columns={bs_df.columns[0]:'Account_year'}, inplace = True) 

    # class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
    is_df = fs.show('cis', show_class = False, show_concept = True).T.reset_index()
    # level_1은 쓸모없는 설명 열 이므로 제거 
    is_df.drop(['level_1'],axis=1, inplace=True)
    # 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
    is_df.columns = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
    is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
    # 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
    is_df.rename(columns={is_df.columns[0]:'Account_year'}, inplace = True) 

    # class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
    cf_df = fs.show('cf', show_class = False, show_concept = True).T.reset_index()
    # level_1은 쓸모없는 설명 열 이므로 제거 
    cf_df.drop(['level_1'],axis=1, inplace=True)
    # 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
    cf_df.columns = cf_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
    cf_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
    # 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
    cf_df.rename(columns={cf_df.columns[0]:'Account_year'}, inplace = True) 
    

    if iter_num == 0 : 
        삼전 = pd.concat([bs_df,is_df,cf_df], axis=1)
        
    elif iter_num == 1 : 
        국민 = pd.concat([bs_df,is_df,cf_df], axis=1)
    
    #if iter_num == 0 : 
    #    국민 = pd.concat([bs_df,is_df,cf_df], axis=1)
        
    #else : 
    #    삼생 = pd.concat([bs_df,is_df,cf_df], axis=1)

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/5 [00:00<?, ?report/s]

Annual reports:   0%|          | 0/5 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/9 [00:00<?, ?report/s]

In [28]:
삼전

,Account_year,ifrs-full_CurrentAssets,ifrs-full_CashAndCashEquivalents,dart_ShortTermDepositsNotClassifiedAsCashEquivalents,entity00126380_udf_BS_201851017339116_CurrentAssets,ifrs-full_CurrentFinancialAssetsAtFairValueThroughProfitOrLossMandatorilyMeasuredAtFairValue,dart_ShortTermTradeReceivable,entity00126380_udf_BS_201710182279121_CurrentAssets,entity00126380_udf_BS_2017101822109437_CurrentAssets,ifrs-full_Inventories,...,entity00126380_udf_CF_20171021102249949_CashFlowsFromUsedInFinancingActivities,entity00126380_udf_CF_2021111023423146_CashFlowsFromUsedInFinancingActivities,dart_RepaymentsOfLongTermBorrowings,ifrs-full_DividendsPaidClassifiedAsFinancingActivities,entity00126380_udf_CF_20171021102253395_CashFlowsFromUsedInFinancingActivities,entity00126380_udf_CF_20201022202135156_StatementOfCashFlowsAbstract,ifrs-full_EffectOfExchangeRateChangesOnCashAndCashEquivalents,ifrs-full_IncreaseDecreaseInCashAndCashEquivalents,dart_CashAndCashEquivalentsAtBeginningOfPeriodCf,dart_CashAndCashEquivalentsAtEndOfPeriodCf
1,"[D210000] Statement of financial position, cur...",유동자산,현금및현금성자산,단기금융상품,단기상각후원가금융자산,단기당기손익-공정가치금융자산,매출채권,미수금,선급비용,재고자산,...,단기차입금의 순증가(감소),장기차입금의 차입,사채 및 장기차입금의 상환,배당금의 지급,비지배지분의 증감,매각예정분류,외화환산으로 인한 현금의 변동,현금및현금성자산의 순증감,기초의 현금및현금성자산,기말의 현금및현금성자산
2,"[D210000] Statement of financial position, cur...",Current assets,Cash and cash equivalents,Short-term financial instruments,Short-term financial assets at amortized cost,Short-term financial assets at fair value thro...,Trade receivables,Non-trade receivables,Prepaid expenses,Inventories,...,Net increase (decrease) in short-term borrowings,Increase in long-term borrowings,Repayment of debentures and long-term borrowings,Dividends paid,Net increase (decrease) in non-controlling int...,Classified as held-for-sale,Effect of exchange rate changes on cash and ca...,Net increase (decrease) in cash and cash equiv...,Cash and cash equivalents at beginning of period,Cash and cash equivalents at end of period
3,20220331,"232,369,082,000,000.0","48,944,982,000,000.0","75,121,426,000,000.0","1,755,225,000,000.0","53,056,000,000.0","44,464,968,000,000.0","5,652,435,000,000.0","3,042,846,000,000.0","47,590,731,000,000.0",...,"213,034,000,000.0","1,000,000,000.0","708,717,000,000.0","461,000,000.0","-202,000,000.0",0.0,"592,355,000,000.0","9,913,567,000,000.0","39,031,415,000,000.0","48,944,982,000,000.0"
4,20211231,"218,163,185,000,000.0","39,031,415,000,000.0","81,708,986,000,000.0","3,369,034,000,000.0","40,757,000,000.0","40,713,415,000,000.0","4,497,257,000,000.0","2,336,252,000,000.0","41,384,404,000,000.0",...,"-2,616,943,000,000.0","58,279,000,000.0","894,749,000,000.0","20,510,350,000,000.0","-27,270,000,000.0","139,000,000.0","1,582,046,000,000.0","9,648,837,000,000.0","29,382,578,000,000.0","39,031,415,000,000.0"
5,20210930,"212,793,019,000,000.0","32,675,040,000,000.0","85,077,401,000,000.0","2,427,925,000,000.0","60,609,000,000.0","42,369,691,000,000.0","4,730,935,000,000.0","2,916,308,000,000.0","37,801,695,000,000.0",...,"-1,534,219,000,000.0","34,324,000,000.0","622,115,000,000.0","18,053,070,000,000.0","-15,622,000,000.0","139,000,000.0","1,708,480,000,000.0","3,292,462,000,000.0","29,382,578,000,000.0","32,675,040,000,000.0"
6,20210630,"191,118,524,000,000.0","30,683,788,000,000.0","77,777,032,000,000.0","2,350,399,000,000.0","49,720,000,000.0","35,624,916,000,000.0","4,312,276,000,000.0","2,595,336,000,000.0","33,592,385,000,000.0",...,"-3,676,341,000,000.0",NaN,"390,466,000,000.0","15,576,303,000,000.0","173,000,000.0","139,000,000.0","801,425,000,000.0","1,301,210,000,000.0","29,382,578,000,000.0","30,683,788,000,000.0"
7,20210331,"209,155,353,000,000.0","41,039,591,000,000.0","87,159,273,000,000.0","3,526,888,000,000.0","59,495,000,000.0","33,962,077,000,000.0","3,828,263,000,000.0","2,660,029,000,000.0","30,619,976,000,000.0",...,"-400,336,000,000.0",0.0,"193,240,000,000.0","458,000,000.0","-289,000,000.0","-551,129,000,000.0","576,996,000,000.0","11,657,013,00

In [25]:
is_df = fs.show('cis', show_class = False, show_concept = True)
is_df.T

,,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
Statement of comprehensive income(Unit: KWR),label_ko,"Ⅰ.순이자이익(주석5,28)",1.이자수익,기타포괄손익-공정가치측정및상각후원가측정,당기손익-공정가치측정금융상품이자수익,2.이자비용,"II.순수수료이익(주석5,29)",1.수수료수익,2.수수료비용,"Ⅲ.순보험손익(주석5,38)",1.보험수익,...,XV.당기총포괄이익,당기순이익의귀속(주석5),지배기업주주지분순이익,비지배지분순이익,당기총포괄이익의귀속,지배기업주주지분총포괄이익,비지배지분총포괄이익,XVI.주당이익(주석37),기본주당이익,희석주당이익
20210101-20211231,"(연결재무제표,)","11,229,572,000,000.0","15,210,878,000,000.0","14,620,490,000,000.0","590,388,000,000.0","-3,981,306,000,000.0","3,625,583,000,000.0","5,323,606,000,000.0","-1,698,023,000,000.0","556,711,000,000.0","16,107,858,000,000.0",...,"4,607,139,000,000.0","4,384,381,000,000.0","4,409,543,000,000.0","-25,162,000,000.0","4,607,139,000,000.0","4,610,549,000,000.0","-3,410,000,000.0",,"11,134.0","10,890.0"
20200101-20201231,"(연결재무제표,)","9,722,274,000,000.0","14,485,747,000,000.0","13,826,382,000,000.0","659,365,000,000.0","-4,763,473,000,000.0","2,958,939,000,000.0","4,527,024,000,000.0","-1,568,085,000,000.0","299,993,000,000.0","14,386,640,000,000.0",...,"3,987,848,000,000.0","3,515,578,000,000.0","3,468,448,000,000.0","47,130,000,000.0","3,987,848,000,000.0","3,966,361,000,000.0","21,487,000,000.0",,"8,843.0","8,730.0"
20190101-20191231,"(연결재무제표,)","9,196,787,000,000.0","14,639,187,000,000.0","13,935,124,000,000.0","704,063,000,000.0","-5,442,400,000,000.0","2,355,004,000,000.0","3,879,247,000,000.0","-1,524,243,000,000.0","299,512,000,000.0","12,317,182,000,000.0",...,"3,465,222,000,000.0","3,313,199,000,000.0","3,311,828,000,000.0","1,371,000,000.0","3,465,222,000,000.0","3,463,567,000,000.0","1,655,000,000.0",,"8,451.0","8,389.0"
20180101-20181231,"(연결재무제표,)","8,904,928,000,000.0","13,734,569,000,000.0","12,986,209,000,000.0","748,360,000,000.0","-4,829,641,000,000.0","2,243,376,000,000.0","3,717,720,000,000.0","-1,474,344,000,000.0","490,116,000,000.0","11,975,070,000,000.0",...,"3,051,464,000,000.0","3,061,946,000,000.0","3,061,191,000,000.0","755,000,000.0","3,051,464,000,000.0","3,050,805,000,000.0","659,000,000.0",,"7,721.0","7,676.0"
20170101-20171231,"(연결재무제표,)","8,246,614,000,000.0","11,919,057,000,000.0",,,"-3,672,443,000,000.0","2,050,024,000,000.0","3,988,250,000,000.0","-1,938,226,000,000.0","593,710,000,000.0","8,970,992,000,000.0",...,"3,479,687,000,000.0","3,343,461,000,000.0","3,311,438,000,000.0","32,023,000,000.0","3,479,687,000,000.0","3,445,285,000,000.0","34,402,000,000.0",,"8,305.0","8,257.0"
20160101-20161231,"(연결재무제표,)","6,402,529,000,000.0","10,021,882,000,000.0",None,None,"-3,619,353,000,000.0","1,584,892,000,000.0","3,150,877,000,000.0","-1,565,985,000,000.0","-117,803,000,000.0","1,201,352,000,000.0",...,"2,165,243,000,000.0","2,190,180,000,000.0","2,143,744,000,000.0","46,436,000,000.0","2,165,243,000,000.0","2,118,829,000,000.0","46,414,000,000.0",,"5,588.0","5,559.0"


In [78]:
# level_1은 쓸모없는 설명 열 이므로 제거 
is_df = fs.show('cis', show_class = False, show_concept = True).T.reset_index()
is_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
is_df.columns = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
is_df.rename(columns={is_df.columns[0]:'Account_year'}, inplace = True) 

In [80]:
bs_df

,Account_year,자산,"Ⅰ.현금및예치금(주석4,6,7,8,39)","II.당기손익-공정가치측정금융자산(주석4,6,8,12)","III.파생금융자산(주석4,6,9)","Ⅳ.상각후원가측정대출채권(주석4,6,10,11)","Ⅴ.투자금융자산(주석4,6,8,12)",Ⅵ.관계기업및공동기업투자(주석13),Ⅶ.유형자산(주석14),Ⅷ.투자부동산(주석14),...,1.자본금,2.신종자본증권,3.자본잉여금,(주석36)4.기타포괄손익누계액,5.매각예정처분자산집단관련,6.이익잉여금,7.자기주식,Ⅱ.비지배지분,자본총계,부채와자본총계
1,20211231,,"31,009,374,000,000.0","66,005,815,000,000.0","3,721,370,000,000.0","417,900,273,000,000.0","104,847,871,000,000.0","448,718,000,000.0","5,239,898,000,000.0","2,514,944,000,000.0",...,"2,090,558,000,000.0","2,838,221,000,000.0","16,940,231,000,000.0","1,047,274,000,000.0","7,671,000,000.0","25,672,815,000,000.0","-1,136,188,000,000.0","833,338,000,000.0","48,293,920,000,000.0","663,895,834,000,000.0"
2,20201231,,"25,608,842,000,000.0","61,035,455,000,000.0","5,545,385,000,000.0","377,166,984,000,000.0","98,695,426,000,000.0","771,435,000,000.0","5,433,554,000,000.0","2,533,539,000,000.0",...,"2,090,558,000,000.0","1,695,988,000,000.0","16,723,589,000,000.0","630,011,000,000.0",,"22,540,616,000,000.0","-1,136,188,000,000.0","857,783,000,000.0","43,402,357,000,000.0","610,719,944,000,000.0"
3,20191231,,"20,837,878,000,000.0","53,549,086,000,000.0","3,190,673,000,000.0","339,684,059,000,000.0","71,782,606,000,000.0","598,240,000,000.0","5,067,377,000,000.0","2,827,988,000,000.0",...,"2,090,558,000,000.0","399,205,000,000.0","17,122,777,000,000.0","348,021,000,000.0",None,"19,709,545,000,000.0","-1,136,188,000,000.0","585,407,000,000.0","39,119,325,000,000.0","518,538,117,000,000.0"
4,20181231,,"20,274,490,000,000.0","50,987,847,000,000.0","2,025,962,000,000.0","319,201,603,000,000.0","61,665,094,000,000.0","504,932,000,000.0","4,272,127,000,000.0","2,119,811,000,000.0",...,"2,090,558,000,000.0",,"17,121,660,000,000.0","177,806,000,000.0",None,"17,282,441,000,000.0","-968,549,000,000.0","9,111,000,000.0","35,713,027,000,000.0","479,588,298,000,000.0"
5,20171231,,"19,817,825,000,000.0",,"3,310,166,000,000.0","290,122,838,000,000.0","66,608,243,000,000.0","335,070,000,000.0","4,201,697,000,000.0","848,481,000,000.0",...,"2,090,558,000,000.0",None,"17,122,228,000,000.0","537,668,000,000.0",None,"15,044,204,000,000.0","-755,973,000,000.0","6,144,000,000.0","34,044,829,000,000.0","436,785,609,000,000.0"
6,20161231,,"17,884,863,000,000.0",None,"3,381,935,000,000.0","265,486,134,000,000.0","45,147,797,000,000.0","1,770,673,000,000.0","3,627,268,000,000.0","755,011,000,000.0",...,"2,090,558,000,000.0",None,"16,994,902,000,000.0","405,329,000,000.0",None,"12,229,228,000,000.0","-721,973,000,000.0","263,359,000,000.0","31,261,403,000,000.0","375,673,656,000,000.0"


In [35]:
# 7월1일~9월31일 / 0401~0631일 동안의 손익계산서는 Drop 
# 분기별 손익계산서는 누적된 것을 사용한다. 다시 말해, 20190630의 보고서는 20190101 기간 부터의 손익계산서.
is_drop_index =  is_df[is_df['Account_year'].str.contains('0701')==True].index
is_drop_index.append(is_df[is_df['Account_year'].str.contains('0401')==True].index)
is_df = is_df.drop(is_drop_index)
is_df['Account_year'] = is_df['Account_year'].apply(lambda x : x.split('-'))
is_df['Account_year'] = is_df['Account_year'].apply(lambda x :x[1])

In [38]:
# string인 'Account_year'를 date time 포맷으로 바꾼다
is_df['Account_year'] = pd.to_datetime(is_df['Account_year'])

In [49]:
# bs, is는 분기로 나오는데 cf만 분기별로 다 나오는 경우 (국민은행)
cf_df.loc[cf_df['Account_year'].str.contains('1231')]

,Account_year,Ⅰ.영업활동으로부터의현금흐름,1.당기순이익,2.손익조정사항,(1)당기손익-공정가치측정금융상품순손익,(2)위험회피목적파생금융상품순손익,(3)파생금융상품공정가치평가충당금환입액,(4)신용손실충당금전입액,(5)투자금융자산순손익,(6)관계기업및공동기업투자손익,...,6.보통주배당지급,7.신종자본증권배당지급,8.신종자본증권발행,9.비지배지분의증감,10.리스부채의원금상환,11.기타,Ⅳ.외화표시현금및현금성자산에대한환율변동효과,Ⅴ.현금및현금성자산의순증감(I+II+III+Ⅳ),Ⅵ.기초현금및현금성자산(주석39),Ⅶ.기말현금및현금성자산(주석39)
2,20210101-20211231,"-6,700,265,000,000.0","4,384,381,000,000.0","5,550,573,000,000.0","-274,515,000,000.0","213,996,000,000.0",,"1,185,133,000,000.0","97,813,000,000.0","-93,526,000,000.0",...,"-981,879,000,000.0","-71,537,000,000.0","1,142,233,000,000.0","-24,145,000,000.0","-253,248,000,000.0","-65,826,000,000.0","158,249,000,000.0","423,368,000,000.0","8,685,092,000,000.0","9,108,460,000,000.0"
6,20200101-20201231,"-11,540,694,000,000.0","3,515,578,000,000.0","4,807,833,000,000.0","-566,447,000,000.0","-52,696,000,000.0","-3,198,000,000.0","1,043,498,000,000.0","-278,805,000,000.0","43,750,000,000.0",...,"-861,092,000,000.0","-22,860,000,000.0","1,296,783,000,000.0","-25,658,000,000.0","-235,498,000,000.0","172,433,000,000.0","-171,805,000,000.0","2,561,367,000,000.0","6,123,725,000,000.0","8,685,092,000,000.0"
10,20190101-20191231,"9,400,706,000,000.0","3,313,199,000,000.0","3,405,812,000,000.0","-438,567,000,000.0","-3,835,000,000.0","282,000,000.0","670,185,000,000.0","-206,192,000,000.0","-16,451,000,000.0",...,"-759,736,000,000.0","-6,513,000,000.0","399,205,000,000.0","574,580,000,000.0","-229,750,000,000.0","134,027,000,000.0","137,019,000,000.0","-519,091,000,000.0","6,642,816,000,000.0","6,123,725,000,000.0"
14,20180101-20181231,"-8,957,618,000,000.0","3,061,946,000,000.0","3,297,857,000,000.0","-104,755,000,000.0","186,029,000,000.0","410,000,000.0","673,694,000,000.0","-99,253,000,000.0","-24,260,000,000.0",...,"-766,728,000,000.0",,,,,"-185,894,000,000.0","-67,950,000,000.0","-1,762,082,000,000.0","8,404,898,000,000.0","6,642,816,000,000.0"
18,20170101-20171231,"-4,836,025,000,000.0","3,343,461,000,000.0","3,237,931,000,000.0",,"-135,363,000,000.0","-1,000,000,000.0","548,244,000,000.0","110,156,000,000.0","-84,274,000,000.0",...,"-497,969,000,000.0",None,None,"-163,658,000,000.0",None,"148,775,000,000.0","-133,240,000,000.0","990,062,000,000.0","7,414,836,000,000.0","8,404,898,000,000.0"
22,20160101-20161231,"1,124,987,000,000.0","2,190,180,000,000.0","1,361,248,000,000.0",None,"69,573,000,000.0","338,000,000.0","539,283,000,000.0","-139,800,000,000.0","-280,838,000,000.0",...,"-378,625,000,000.0",None,None,,None,"-38,786,000,000.0","89,142,000,000.0","-43,083,000,000.0","7,457,919,000,000.0","7,414,836,000,000.0"


In [56]:
pd.merge(bs_df,is_df)

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

# 계정과목 대응표 작성후 가져오기
핵심이 되는 계정과목들과 그에 대응하는 ifrs 코드를 따로 엑셀로 만들어 두었고, 이를 불러온다. 

In [ ]:
계정과목대응표 = pd.read_excel('대응표.xlsx', index_col= 0)

In [ ]:
blank = pd.DataFrame(columns = list(계정과목대응표['기준']))

In [ ]:
import numpy as np

In [ ]:
for order, code in enumerate(list(계정과목대응표['ifrs_code'])) : 
    try :
        blank.iloc[:,order] = 삼전[code] 
    except : 
        
        for alternative in list(계정과목대응표.loc[str(code),:]) : 

            # alternative에 대응하는 데이터 프레임속 칼럼명을 반환
            find_result = 삼전.isin([str(alternative)]).any()
            find_result = find_result[find_result==True]
            
            try : 
                # 주소변환 
                
                blank.iloc[:,order] = 삼전[find_result.index].iloc[:,0]
                print(f"대체 성공! {code}를 {alternative}로")
                break
                
            except :
                print(f"대체 실패! {code}를 {alternative}로")
            
            pass  ## pass로 인해 혹시 라도 매칭이 안되는 부분을 blank처리하여 건너뛰게 된다. 
            

In [ ]:
blank.to_excel('show.xlsx')

# 기업개황 추가 

In [223]:
dart2 = OpenDartReader('7db60698e63bd621a006025500fe0436ee65cb47') 

In [226]:
dart2.company('00164742')

{'status': '000',
 'message': '정상',
 'corp_code': '00164742',
 'corp_name': '현대자동차(주)',
 'corp_name_eng': 'HYUNDAI MOTOR CO',
 'stock_name': '현대자동차',
 'stock_code': '005380',
 'ceo_nm': '정의선, 장재훈, 이동석(각자 대표이사)',
 'corp_cls': 'Y',
 'jurir_no': '1101110085450',
 'bizr_no': '1018109147',
 'adres': '서울특별시 서초구  헌릉로 12',
 'hm_url': 'www.hyundai.com',
 'ir_url': '',
 'phn_no': '02-3464-1114',
 'fax_no': '02-3464-8719',
 'induty_code': '30121',
 'est_dt': '19671229',
 'acc_mt': '12'}

In [227]:
blank

,유동자산금액,비유동자산금액,자산총계금액,유동부채금액,비유동부채금액,부채총계금액,자본총계금액,부채자본총계금액,매출액,매출원가금액,...,영업이익손실금액,이자수익,기타영업외수익금액,법인세차감전순이익,당기순이익,영업활동현금흐름금액,투자활동현금흐름금액,재무활동현금흐름금액,기초현금자산금액,기말현금자산금액
1,유동자산,현금및현금성자산,자산총계,유동부채,비유동부채,부채총계,자본총계,자본과부채총계,수익(매출액),매출원가,...,영업이익(손실),금융수익,기타이익,법인세비용차감전순이익(손실),당기순이익(손실),영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,기초현금및현금성자산,기말현금및현금성자산
2,Current assets,Cash and cash equivalents,Total assets,Current liabilities,Non-current liabilities,Total liabilities,Total equity,Total equity and liabilities,Revenue(Sales),Cost of sales,...,Operating income(loss),Finance income,Other gains,Profit (loss) before tax,Profit (loss),Cash flows from (used in) operating activities,Cash flows from (used in) investing activities,Cash flows from (used in) financing activities,Cash and cash equivalents at beginning of period,Cash and cash equivalents at end of period
3,"30,147,945,276.0","6,183,380,605.0","51,602,974,173.0","7,363,854,860.0","2,382,777,092.0","9,746,631,952.0","41,856,342,221.0","51,602,974,173.0","61,382,432,205.0","47,831,239,592.0",...,"7,933,450,389.0","185,676,724.0","527,977,556.0","8,460,577,268.0","7,572,126,967.0","8,953,404,540.0","-4,884,653,034.0","-2,595,036,592.0","4,754,074,224.0","6,183,380,605.0"
4,"26,714,884,501.0","4,754,074,224.0","42,743,113,508.0","7,090,978,482.0","1,149,275,296.0","8,240,253,778.0","34,502,859,730.0","42,743,113,508.0","39,274,889,541.0","33,069,025,876.0",...,"1,654,132,531.0","175,538,403.0","257,325,024.0","1,361,359,874.0","1,120,573,601.0","2,367,042,777.0","-3,321,747,680.0","2,414,652,339.0","3,523,688,591.0","4,754,074,224.0"
5,"19,479,418,263.0","3,523,688,591.0","38,044,902,802.0","3,449,643,267.0","1,222,213,414.0","4,671,856,681.0","33,373,046,121.0","38,044,902,802.0","34,428,726,931.0","29,440,042,913.0",...,"619,874,442.0","164,536,317.0","315,211,307.0","745,745,780.0","831,054,762.0","249,578,130.0","793,010,534.0","-174,456,157.0","2,707,874,313.0","3,523,688,591.0"
6,"16,875,306,764.0","2,707,874,313.0","36,450,622,701.0","2,431,957,777.0","1,438,483,690.0","3,870,441,467.0","32,580,181,234.0","36,450,622,701.0","25,683,997,231.0","24,789,264,544.0",...,"-3,516,965,900.0","212,032,314.0","203,324,187.0","-4,541,883,426.0","-4,158,428,011.0","-1,798,581,064.0","641,409,275.0","-19,145,240.0","3,836,252,585.0","2,707,874,313.0"
7,"22,622,966,544.0","3,836,252,585.0","41,284,656,528.0","3,059,994,779.0","1,293,552,043.0","4,353,546,822.0","36,931,109,706.0","41,284,656,528.0","27,006,640,223.0","24,788,247,730.0",...,"-1,785,617,407.0","211,097,070.0","84,403,160.0","-4,166,149,326.0","-3,856,012,921.0","1,662,599,181.0","-1,614,214,626.0","-241,615,320.0","3,948,512,520.0","3,836,252,585.0"
8,"23,599,897,057.0","3,948,512,520.0","44,740,846,909.0","2,463,583,908.0","1,266,245,771.0","3,729,829,679.0","41,011,017,230.0","44,740,846,909.0","27,223,309,387.0","24,095,572,728.0",...,"-647,162,640.0","244,316,363.0","429,949,965.0","-523,577,482.0","-249,972,876.0","-222,760,108.0","161,402,754.0","-966,963,280.0","4,982,762,984.0","3,948,512,520.0"


# 클래스로 정리

In [48]:
class UltraCralwer : 
    def __init__(self,start_year, end_year, quarter, ) : 
        self.target_accout  = 
        self.target_basicinfo =         
        
    def 

SyntaxError: invalid syntax (Temp/ipykernel_5668/3795466177.py, line 3)

쉣;;;;; x같이 다다르다. 어떻게 해야할까?

## Solution. Try1 
아래 실험은 FastText 임베딩 기술을 사용하여, 표준계정과목들과 각 기업들이 사용하는 계정 과목들의 유사도를 계산하고, 유사도가 가장 높은 것으로 매칭시키는 실험이다. 이 실험을 통해 표준화가 자동적으로 될 수 있는지 알아본다.  

참고 
1. 모델 임포팅 방법 : https://inahjeon.dev/fasttext/
2. wiki Docs https://wikidocs.net/22883
3. gensim  https://radimrehurek.com/gensim/models/fasttext.html   
4. Facebook 공식 문서 https://fasttext.cc/docs/en/crawl-vectors.html

In [ ]:
from gensim import models

model = models.fasttext.load_facebook_model('C:\\Users\\user\\Downloads\\cc.en.300.bin.gz')

In [ ]:
model.wv.similarity('iron','python')

0.11623385

1. 한국 표준계정과목의 영문명을 불러온다.

In [ ]:
import pandas as pd
standard = pd.read_excel('./한국표준계정과목_비교.xlsx')

In [ ]:
standard['대응하는 영어 명칭']

0                                     Assets [abstract]
1                                        Current assets
2                             Cash and cash equivalents
3          Government grants, cash and cash equivalents
4     Short-term deposits not classified as cash equ...
5                   Trade and other current receivables
6                                    Current tax assets
7                                           Inventories
8                        Other current financial assets
9                                  Other current assets
10    Current non-cash assets pledged as collateral ...
11    Non-current assets or disposal groups classifi...
12                                   Non-current assets
13    Long-term deposits not classified as cash equi...
14    Long-term trade and other non-current receivab...
15                        Property, plant and equipment
16                                  Investment property
17                                      Goodwill

2. 삼성의 연결재무제표 - 재무상태표를 갖고온다.

In [ ]:
import dart_fss as dart

# Open DART API KEY 설정
api_key='7db60698e63bd621a006025500fe0436ee65cb47'
dart.set_api_key(api_key=api_key)

## 삼성, 현차 
corp_code = '00126380'   

# 2019년 01월 01일 부터 연결재무제표 검색
# fs = samsung.extract_fs(bgn_de='20120101') 와 동일
fs = dart.fs.extract(corp_code=corp_code, bgn_de='20190101')


# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df = fs.show('bs', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
bs_df.drop(['level_1'],axis=1, inplace=True)
bs_df.columns = bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
bs_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
bs_df.rename(columns={bs_df.columns[0]:'Account_year'}, inplace = True) 

#삼전 = pd.concat([bs_df,is_df,cf_df], axis=1)        

Annual reports:   0%|          | 0/4 [00:00<?, ?report/s]

In [212]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df = fs.show('bs', show_class = False, show_concept = True).T.reset_index()

In [ ]:
pd.DataFrame(columns=standard['대응하는 영어 명칭']).T

""
대응하는 영어 명칭
Assets [abstract]
Current assets
Cash and cash equivalents
"Government grants, cash and cash equivalents"
Short-term deposits not classified as cash equivalents
Trade and other current receivables
Current tax assets
Inventories
Other current financial assets


유사도 계산.

In [ ]:
# 이중 루프 
# Level3의 표준계정과목에 대해 삼성의 계정과목들을 비교해 가장 표준계정과목과 가장 유사한 삼성의 계정과목들을 뽑아낸다. 

record = pd.DataFrame(columns=['표준계정과목','삼성계정과목','유사도'])

for iter_num, standard_acc_title in enumerate(standard['대응하는 영어 명칭']) : 
    old_score = 0
    
    for samsung_acc_title in bs_df.iloc[1,1:] : 
        # similarity score
        score = model.wv.similarity(standard_acc_title, samsung_acc_title)
        
        if score > old_score : 
            most_similar = samsung_acc_title
            best_score = score
            
        else : 
            continue
            
        # 비교의 기준이 되는 old_score 업데이트
        old_score = score
        
        
    # 표준 계정과목, 삼성 계정과목, 유사도
    record.loc[iter_num] = [standard_acc_title, most_similar, best_score]

In [ ]:
model.wv.similarity('Current assets', 'Current assets')

0.99999994

In [ ]:
record.to_excel('삼성계정과목 테스트결과.xlsx')

In [ ]:
for i in bs_df.iloc[1,1:] : 
    print(i)

Current assets
Cash and cash equivalents
Short-term financial instruments
Short-term financial assets at amortized cost
Short-term financial assets at fair value through profit or loss
Trade receivables
Non-trade receivables
Prepaid expenses
Inventories
Other current assets
Assets held-for-sale
Non-current assets
Financial assets at fair value through other comprehensive income
Financial assets at fair value through profit or loss
Investment in associates and joint ventures
Property, plant and equipment
Intangible assets
Net defined benefit assets
Deferred income tax assets
Other non-current assets
Total assets
Current liabilities
Trade payables
Short-term borrowings
Other payables
Advances received
Withholdings
Accrued expenses
Current income tax liabilities
Current portion of long-term liabilities
Provisions
Other current liabilities
Liabilities held-for-sale
Non-current liabilities
Debentures
Long-term borrowings
Long-term other payables
Net defined benefit liabilities
Deferred inco